Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Credit Card Fraud Detection: Proving Causal Generation in Causal Cooperative Network (CCNet)

## Introduction

This tutorial delves into the application of the Cooperative Network (CCNet), enhanced with the CounterGenerate method, to address challenges associated with imbalanced datasets in the domain of credit card fraud detection. We aim to leverage CCNet not just as a data augmentation tool but as a causal model capable of generating realistic training data distributions through counterfactual scenarios. By proving that \(Y\) (outcomes) and \(E\) (explanations) are necessary and sufficient causes within our model, we intend to demonstrate CCNet's ability to perform causal generation, thereby enhancing the diversity, realism, and balance of the training data. This, in turn, is expected to improve the robustness and accuracy of predictive models designed to identify fraudulent transactions.

## Tutorial Goals

The objectives of this tutorial are to guide you through the innovative use of causal data generation techniques and validate the effectiveness of these methods in a practical setting:

### Counterfactual Scenarios
- **Manipulate \(Y\), Maintain \(E\)**: Delve into the dynamics of manipulating the outcome variable \(Y\) while keeping the explanation variable \(E\) constant to create counterfactually augmented datasets. These datasets will represent balanced and redistributed conditions, allowing us to test the hypothesis that they can accurately mirror real-world data distributions. This approach underscores the causal influence of \(Y\) in the data generation process and tests the robustness of \(E\) as a stable explanatory factor.

### Classifier Training and Causal Model Validation
- **Classifier Training**: Engage in the training of classifiers using datasets that have been created under specific causal assumptions to compare their performance metrics:
  - **First Classifier**: Trained on the **original dataset**, serving as the baseline for performance comparison.
  - **Second Classifier**: Trained on the **redistributed fraud outcome dataset**, designed to maintain the same frequency of fraud as the original dataset. This approach aims to test the model's effectiveness under different distribution conditions while expecting to achieve similar prediction accuracy to the first classifier.
  - **Third Classifier**: Trained on a **balanced fraud outcome dataset** where the fraud rate is intentionally reduced to half of its original rate, representing a more severe case of imbalance. This setup challenges the classifier to maintain high detection capabilities under significantly skewed conditions.
- **Performance Metrics**: Utilize key metrics such as the F1 score to evaluate and substantiate the causal generation effectiveness of CCNet. 
- **Counter Generation's Role in Imbalance Mitigation**: Explore how CCNet's counter generation capabilities can be leveraged to synthetically enhance the dataset. By generating realistic, counterfactual instances of both fraudulent and non-fraudulent transactions, CCNet aims to provide a richer, more balanced dataset that helps models learn more effective patterns for fraud detection without the typical bias introduced by skewed data distributions.

### Testing and Empirical Validation
- **Model Evaluation**: Conduct rigorous evaluations of models across all datasets using a consistent test set to ensure fair comparisons.
- **Causal Effectiveness Analysis**: Analyze whether the counterfactually generated data provides a statistically significant improvement in model performance, thereby supporting the hypothesis of effective causal generation.

## Conclusion

By the end of this tutorial, participants will not only comprehend the theoretical framework behind using CCNet for causal data generation but will also witness firsthand the practical benefits of this approach in enhancing model performance for fraud detection tasks. This exploration is intended to solidify the understanding of CCNet as a powerful tool for generating realistic and balanced datasets through principled causal mechanisms.


In [1]:
import sys
path_append = "../../"
sys.path.append(path_append)  # Go up one directory from where you are.


In [2]:
# https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_7_DeepLearning/FeedForwardNeuralNetworks.html
import pandas as pd 

file_name = 'credit_card_fraud_detection'
dataroot = path_append + f"../data/{file_name}/creditcard.csv"
df = pd.read_csv(dataroot)
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [3]:
print('No Frauds', round(df['Class'].value_counts()[0] / len(df) *100,2), '%of the dataset')
print('Frauds', round(df['Class'].value_counts()[1] / len(df) *100,2), '%of the dataset')

No Frauds 99.83 %of the dataset
Frauds 0.17 %of the dataset


In [4]:
import torch
from torch.utils.data import Dataset
from tools.preprocessing.data_frame import auto_preprocess_dataframe

# convert column name time to v0
df.rename(columns={'Time': 'V0'}, inplace=True)
df, description = auto_preprocess_dataframe(df, target_columns=['Class'])

# Calculate the number of features and classes
num_features = description['num_features']
num_classes = description['num_classes']

print(num_features, num_classes)

,Min,Max,Mean,Std,Null Count,Scaled,Encoded
V0,-2.215465,1.822074,-9.580116e-17,1.109630,0,Minmax,None
V1,-56.407510,2.454930,1.556769e-17,1.958696,0,None,None
V2,-72.715728,22.057729,-2.235360e-17,1.651309,0,None,None
V3,-48.325589,9.382558,-8.941442e-17,1.516255,0,None,None
V4,-5.683171,16.875344,0.000000e+00,1.415869,0,None,None
V5,-113.743307,34.801666,2.714366e-17,1.380247,0,None,None
V6,-26.160506,73.301626,6.785916e-18,1.332271,0,None,None
V7,-43.557242,120.589494,7.185087e-18,1.237094,0,None,None
V8,-73.216718,20.007208,-4.291094e-18,1.194353,0,None,None
V9,-13.434066,15.594995,1.347204e-18,1.098632,0,None,None


30 1


In [5]:

# Defining the labeled and unlabeled dataset classes
class LabeledDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        if isinstance(self.x[index], torch.Tensor):
            vals = self.x[index]
        else:
            vals = torch.tensor(self.x[index], dtype=torch.float32)
        if isinstance(self.y[index], torch.Tensor):
            label = self.y[index]
        else:
            label = torch.tensor(self.y[index], dtype=torch.float32)
        return vals, label

#### Dataset Splitting for Training and Testing

The original dataset is split into training and testing parts to evaluate the model's performance accurately. This step is crucial for validating the effectiveness of the training on unseen data.


In [6]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets for model evaluation
df_train, df_test = train_test_split(df, test_size=0.5, shuffle=False)

fraud_counts = df_train['Class'].value_counts(normalize=True)[1]
non_fraud_counts = df_train['Class'].value_counts(normalize=True)[0]
train_fraud_rate = fraud_counts/(fraud_counts+non_fraud_counts)
print("fraud_rate: ", train_fraud_rate)

X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:, -1:].values

# Labeled datasets for supervised learning tasks
trainset = LabeledDataset(X_train, y_train)  # Corrected to include training data
testset = LabeledDataset(X_test, y_test)     # Test set with proper labels

# Printing the shapes of the datasets for verification
print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

fraud_rate:  0.0018890051473634685
Labeled Trainset Shape: 142403, 30
Labeled Testset Shape: 142404, 30


#### Initial Setup and Model Configuration

This section initializes the environment by setting a fixed random seed to ensure reproducibility of results. It imports necessary configurations and initializes model parameters with specific configurations. The model specified here is set to have no core model but uses a 'tabnet' encoder model for data processing, which is particularly tailored for structured or tabular data like credit card transactions.


In [7]:
# Set a fixed random seed for reproducibility of experiments
from nn.utils.init import set_random_seed
set_random_seed(0)

# Importing configuration setups for ML parameters and data
import torch
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub

# Configuration for the data handling, defining dataset specifics and the task type
data_config = DataConfig(dataset_name='credit_card_fraud_detection', task_type='binary_classification', obs_shape=[num_features], label_size=num_classes, explain_size=num_features-num_classes)

# Initializing ML parameters without a core model and setting the encoder model to 'tabnet' with specific configurations
ml_params = MLParameters(ccnet_network='tabnet', encoder_network='none')

# Setting training parameters and device configuration
ml_params.training.num_epoch = 10
ml_params.model.ccnet_config.num_layers = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Create a TrainerHub instance to manage training and data processing
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=False, use_wandb=True)


Trainer Name: causal_trainer


ModelParameters Parameters:


,ccnet_config,ccnet_network,encoder_config,encoder_network
0,See details below,tabnet,None,none



Detailed ccnet_config Configuration:


,ccnet_config_model_name,ccnet_config_num_layers,ccnet_config_d_model,ccnet_config_dropout,ccnet_config_obs_shape,ccnet_config_reset_pretrained
0,tabnet,4,256,0.05,[30],False


TrainingParameters Parameters:


...

,clip_grad_range,decay_rate_100k,learning_rate,max_grad_norm,scheduler_type
0,None,0.05,0.001,1.0,exponential


AlgorithmParameters Parameters:


,enable_diffusion,error_function,reset_pretrained
0,False,mse,False


DataConfig Parameters:


,dataset_name,task_type,obs_shape,label_size,explain_size,explain_layer,state_size,show_image_indices
0,credit_card_fraud_detection,binary_classification,[30],1,29,tanh,None,None


In [8]:
trainer_hub.train(trainset)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junhopark. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\credit_card_fraud_detection\causal-learning)... Done. 0.0s


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

...

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

#### Data Loading and Counter Generation

This section deals with loading the unlabelled dataset, processing it through the trained model to create synthetic data. This data augmentation step is crucial for models that benefit from larger datasets, such as in fraud detection scenarios.


In [9]:
import torch
import torch.nn.functional as F

len_trainset = len(trainset)
causal_model = trainer_hub.ccnet
# Initialize the recreated dataset container

# Assuming X_train and y_train are your features and binary labels respectively
train_X = torch.tensor(X_train, dtype=torch.float32)
train_y = torch.tensor(y_train, dtype=torch.float32)  # Ensure labels are in long dtype for one_hot

# Convert binary labels to one-hot encoding
# PyTorch's one_hot requires the number of classes, which is 2 for binary labels
train_rand_y = torch.rand_like(train_y)  
balanced_train_y = torch.where(torch.rand_like(train_rand_y) > 0.5, torch.ones_like(train_y), torch.zeros_like(train_y)).float()
redistributed_train_y = torch.where(torch.rand_like(train_rand_y) < train_fraud_rate, torch.ones_like(train_y), torch.zeros_like(train_y)).float()

# Now use the one-hot encoded labels for generating data
balanced_data = causal_model.counter_generate(train_X.to(device), balanced_train_y.to(device))
redistributed_data = causal_model.counter_generate(train_X.to(device), redistributed_train_y.to(device))

# Assuming recreated_dataset is a PyTorch tensor already available in your context
balanced_trainset = LabeledDataset(balanced_data.clone().detach().cpu(), balanced_train_y)
redistributed_trainset = LabeledDataset(redistributed_data.clone().detach().cpu(), redistributed_train_y)

print(f"Original Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Balanced Trainset Shape: {len(balanced_trainset)}, {balanced_trainset.x.shape[1]}")
print(f"Redistributed Trainset Shape: {len(redistributed_trainset)}, {redistributed_trainset.x.shape[1]}")

print(f"Original Trainset Fraud Rate: {trainset.y.sum().item()/len(trainset)}")
print(f"Balanced Trainset Fraud Rate: {balanced_trainset.y.sum().item()/len(balanced_trainset)}")
print(f"Redistributed Trainset Fraud Rate: {redistributed_trainset.y.sum().item()/len(redistributed_trainset)}")



Original Trainset Shape: 142403, 30
Balanced Trainset Shape: 142403, 30
Redistributed Trainset Shape: 142403, 30
Original Trainset Fraud Rate: 0.0018890051473634685
Balanced Trainset Fraud Rate: 0.49803726045097363
Redistributed Trainset Fraud Rate: 0.001966250711010302


#### Data Preparation for Model Training

After synthetic data generation, this section separates the data and labels for training purposes, preparing them for use in machine learning models to ensure proper supervision and evaluation.


In [10]:
from nn.tabnet import EncoderTabNet as TabNet
from tools.setting.ml_params import ModelConfig, CCNetConfig

class Classifier(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=3, hidden_size=256):
        super(Classifier, self).__init__()
        
        model_config = ModelConfig('tabnet')
        model_config.num_layers = num_layers
        model_config.d_model = hidden_size
        network_config = CCNetConfig(model_config, "Classifier", input_size, output_size, 'sigmoid')
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        ## Add TabNet layers
        layers.append(TabNet(network_config))

        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

#### Training Supervised Models

This section outlines the process of training supervised learning models using both original and synthetic datasets. The `train_supervised_model` function is designed to iterate through the dataset, perform forward passes, compute loss, and update model weights using backpropagation.


In [11]:
def train_classifier(model, dataset, num_epoch=4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Ensure reproducibility by resetting the random seed
    # Create DataLoader for batch processing
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
    # Training loop
    len_trainset = len(dataset)
    for epoch in range(num_epoch):  # Train for 2 epochs as an example
        sum_loss = 0
        for i, (data, label) in enumerate(trainloader):
            data = data.to(device).clone().detach()
            label = label.to(device).float()
            # Perform forward pass
            output = model(data)
            # Compute loss
            loss = torch.nn.functional.binary_cross_entropy(output, label)
            # Backward pass to compute gradients
            loss.backward()
            # Update weights
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()
            sum_loss += loss.item()
            
        print(f"Epoch {epoch+1}, Loss: {sum_loss/len_trainset}")


#### Model Training Using Recreated and Original Datasets

Models are trained using both datasets generated through the Data Augmentation process and the original dataset. This comparison helps to determine the effectiveness of the synthetic data in improving model performance.


In [12]:
# Initialize and train a model on the recreated dataset
model_trained_on_balanced = Classifier(input_size=num_features, output_size=num_classes).to(device)
train_classifier(model_trained_on_balanced, balanced_trainset)

Epoch 1, Loss: 8.344029060924992e-05
Epoch 2, Loss: 5.8794276960377206e-06
Epoch 3, Loss: 6.204505377750673e-06
Epoch 4, Loss: 3.056491297529599e-06


In [13]:
# Initialize and train a model on the recreated dataset
model_trained_on_redistributed = Classifier(input_size=num_features, output_size=num_classes).to(device)
train_classifier(model_trained_on_redistributed, redistributed_trainset)

Epoch 1, Loss: 7.524845924980525e-05
Epoch 2, Loss: 3.970135570187057e-06
Epoch 3, Loss: 3.962471281785826e-06
Epoch 4, Loss: 5.3331147776981935e-06


In [14]:

# Initialize and train a model on the original dataset
model_trained_on_original = Classifier(input_size=num_features, output_size=num_classes).to(device)
train_classifier(model_trained_on_original, trainset)

Epoch 1, Loss: 0.0001663089547827994
Epoch 2, Loss: 5.052826424942387e-05
Epoch 3, Loss: 4.558326198387667e-05
Epoch 4, Loss: 4.1145633817825555e-05


#### Evaluating Model Performance

After training, the models are evaluated using the F1 score, a harmonic mean of precision and recall, which is particularly useful in the context of imbalanced datasets like fraud detection. This step is critical for assessing the quality of the models trained on different types of data.


In [15]:
from sklearn.metrics import f1_score

def get_f1_score(model, testset, batch_size=256):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []
    # DataLoader for testing
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

    # No gradient computation needed during inference
    with torch.no_grad():
        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            # Process output for binary classification
            predicted = (output.squeeze() > 0.5).long()
            y_true.extend(label.squeeze().long().cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute and return the F1 score
    score = f1_score(y_true, y_pred, average='binary')
    return score

# Calculate F1 scores for both models
f1_score_original = get_f1_score(model_trained_on_original, testset)
f1_score_balanced = get_f1_score(model_trained_on_balanced, testset)
f1_score_redistributed = get_f1_score(model_trained_on_redistributed, testset)

# Output the results
print("F1 score of the classifier trained on the original data: ", f1_score_original)
print("F1 score of the classifier trained on the balanced data: ", f1_score_balanced)
print("F1 score of the classifier trained on the redistributed data: ", f1_score_redistributed)


F1 score of the classifier trained on the original data:  0.8205128205128205
F1 score of the classifier trained on the balanced data:  0.8262910798122066
F1 score of the classifier trained on the redistributed data:  0.8246445497630331
